In [2]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.3 MB/s eta 0:00:00 0:00:01


In [ ]:
#run this in command line to start the server
#python -m llmtuner.api.app --model_name_or_path /workspace/LLaMA-Factory/qwen-1.5-0.5B-tool --template default


In [9]:
import os
import json
from openai import OpenAI
from typing import Sequence

os.environ["OPENAI_BASE_URL"] = "http://0.0.0.0:8000/v1" # Replace with local host IP
os.environ["OPENAI_API_KEY"] = "0"

def calculate_gpa(grades: Sequence[str], hours: Sequence[int]) -> float:
    grade_to_score = {"A": 4, "B": 3, "C": 2}
    total_score, total_hour = 0, 0
    for grade, hour in zip(grades, hours):
        total_score += grade_to_score[grade] * hour
        total_hour += hour
    return total_score / total_hour

tool_map = {
    "calculate_gpa": calculate_gpa
}

if __name__ == "__main__":
    client = OpenAI()
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate_gpa",
                "description": "Calculate GPA based on course grades and credit hours",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "grades": {"type": "array", "items": {"type": "string"}, "description": "Course Grades"},
                        "hours": {"type": "array", "items": {"type": "integer"}, "description": "course hours"},
                    },
                    "required": ["grades", "hours"],
                },
            },
        }
    ]
    messages = []
    messages.append({"role": "user", "content": "My grades are A, A, B, C, and the credit hours are 3, 4, 3, 2"})
    result = client.chat.completions.create(messages=messages, model="yi-agent-6b", tools=tools)
    tool_call = result.choices[0].message.tool_calls[0].function
    name, arguments = tool_call.name, json.loads(tool_call.arguments)
    messages.append({"role": "function", "content": json.dumps({"name": name, "argument": arguments}, ensure_ascii=False)})
    tool_result = tool_map[name](**arguments)
    messages.append({"role": "tool", "content": json.dumps({"gpa": tool_result}, ensure_ascii=False)})
    result = client.chat.completions.create(messages=messages, model="yi-agent-6b", tools=tools)
    print(result.choices[0].message.content)

Based on your grades and credit hours, your GPA would be approximately 3.4166666666666665.


In [10]:
import gc, inspect, json, re
import xml.etree.ElementTree as ET
from functools import partial
from typing import get_type_hints

import transformers
import torch

from langchain.chains.openai_functions import convert_to_openai_function
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field, validator

In [11]:
class BookRecommendation(BaseModel):
    """Provides book recommendations based on specified interest."""
    interest: str = Field(description="question of user interest about a book.")
    recommended_book: str = Field(description="answer to recommend a book")

    @validator("interest")
    def interests_must_not_be_empty(cls, field):
        if not field:
            raise ValueError("Interest cannot be empty.")
        return field
     

In [12]:
class Joke(BaseModel):
    """Get a joke that includes the setup and punchline"""
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

In [13]:
class SongRecommendation(BaseModel):
    """Provides song recommendations based on specified genre."""
    genre: str = Field(description="question to recommend a song.")
    song: str = Field(description="answer to recommend a song")

    @validator("genre")
    def genre_must_not_be_empty(cls, field):
        if not field:
            raise ValueError("genre cannot be empty.")
        return field

In [15]:
sr = convert_pydantic_to_openai_function(SongRecommendation)


In [16]:
sr

{'name': 'SongRecommendation',
 'description': 'Provides song recommendations based on specified genre.',
 'parameters': {'type': 'object',
  'properties': {'genre': {'description': 'question to recommend a song.',
    'type': 'string'},
   'song': {'description': 'answer to recommend a song', 'type': 'string'}},
  'required': ['genre', 'song']}}

In [17]:
client = OpenAI()
tools = [
    {
        "type": "function",
        "function": sr,
    }
]
messages = []
messages.append({"role": "user", "content": "Recommend some hiphop song"})
result = client.chat.completions.create(messages=messages, model="yi-agent-6b", tools=tools)
tool_call = result.choices[0].message.tool_calls[0].function

In [18]:
tool_call

Function(arguments='{"genre": "hiphop"}', name='SongRecommendation')